In [64]:
import pandas as pd
import numpy as np
from datetime import datetime

#read data in 
df = pd.read_csv('phone_data_clean.csv')

In [65]:
#set capacity from 1-10
capacity = np.arange(2,11)

In [66]:
def rho(lambda_hour,c,mew):
    return lambda_hour/(c*mew)

def factorial(n):
    fact = 1
    if n==0:
        fact = 1
    else:
        for i in range(1,n+1):
            fact = fact * i
    return fact

def p_0(c,lambda_hour,mew,rho):
    #first_term = sum(((1/factorial(c-1))*((c*rho)**(c-1))) for c in range(1,c+1))
    #second_term = (1/factorial(c))*((rho*c)**c)*(1/(1-rho))
    first_term = sum(((1/factorial((c-1)))*((lambda_hour/mew)**(c-1))) for c in range(1,c+1))
    second_term = (1/factorial(c))*((lambda_hour/mew)**c)*((mew*c)/(mew*c - lambda_hour))
    p_0 = 1/(first_term + second_term)
    return p_0

def l_q(c,lambda_hour,mew,p_0,rho):
    #factor = (1/factorial(c))*((lambda_hour/mew)**c)*(rho/(1 - rho)**2)
    factor = (1/factorial(c-1))*((lambda_hour/mew)**c)*((mew*lambda_hour)/(c*mew - lambda_hour)**2)
    return factor*p_0

def l_s(l_q,lambda_hour,mew):
    return l_q + (lambda_hour/mew)

def l_w(c,mew,lambda_hour):
    return ((c*mew)/(c*mew - lambda_hour))

def w_q(l_q,lambda_hour):
    return l_q/lambda_hour

def w_s(l_s,lambda_hour):
    return l_s/lambda_hour


In [67]:
c = 3
mew = 1/2
lambda_hour = 1
rho = rho(lambda_hour, c, mew)
p_0  = p_0(c,lambda_hour,mew,rho)
l_q = l_q(c,lambda_hour,mew,p_0,rho)
w_q = w_q(l_q,lambda_hour)
print(rho,w_q)

0.6666666666666666 0.8888888888888888


In [68]:
df['Start time'] = pd.to_datetime(df['Start time'], format="%Y-%m-%d %H:%M:%S")
no_days = len(df['Start time'].dt.normalize().value_counts())
df = df.groupby(df['Call_ID'])

In [69]:
arrival_rate = {}
for i in range(7,21):
    arrival_rate[i] = 0

In [70]:
for name, group in df:
    arrival_rate[(group['Start time'].dt.hour).min()] += (1/no_days)

In [71]:
print(arrival_rate)

{7: 2.227272727272726, 8: 43.54545454545527, 9: 31.909090909091308, 10: 24.45454545454564, 11: 29.90909090909125, 12: 27.363636363636633, 13: 24.000000000000174, 14: 27.1363636363639, 15: 24.272727272727455, 16: 17.36363636363635, 17: 9.772727272727286, 18: 1.5909090909090902, 19: 0, 20: 0}


5798
